In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [104]:
csv_file = 'C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Necessidades Humanas Básicas/nascidos_vivos/nascidos_vivos_2021.csv'
output_directory = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Necessidades Humanas Básicas/nascidos_vivos/base_2021_small/"

path_dicio = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Dicionario, Documentanção e Codigos/dicionario_regioes.csv"
dicio = pd.read_csv(path_dicio)

In [32]:
chunksize = 500000 
reader = pd.read_csv(csv_file, chunksize=chunksize, dtype=dtypes, sep=";") # Esse dtypes foi uma gambiarra que eu pensei a usar 

for i, chunk in enumerate(reader):
    parquet_file = f'{output_directory}chunk_{i}.parquet'
    table = pa.Table.from_pandas(chunk)
    pq.write_table(table, parquet_file)
    del table

In [105]:
import pandas as pd
files = []
for file in range(6):
    path = f"C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Necessidades Humanas Básicas/nascidos_vivos/base_2021_small/chunk_{file}.parquet"
    data = pd.read_parquet(path)
    data["contagem"] = 1
    data = data[["IDADEMAE", "CODMUNRES", "ESTCIVMAE", "contagem"]]
    data.rename(columns={"IDADEMAE": "idade", "CODMUNRES": "cod_municipio_6", "ESTCIVMAE": "estado_civil", "contagem": "nascidos_vivos"}, inplace=True)
    files.append(data)
    print(f"Arquivo {file} processado")
    del data
nascidos_vivos = pd.concat(files)

nascidos_vivos = nascidos_vivos[nascidos_vivos["cod_municipio_6"].isin(lista_mun)]
nascidos_vivos["idade"] = nascidos_vivos["idade"].fillna(99999)
nascidos_vivos["idade"] = nascidos_vivos["idade"].astype(int)
nascidos_vivos["nascidos_vivos"] = 1

Arquivo 0 processado
Arquivo 1 processado
Arquivo 2 processado
Arquivo 3 processado
Arquivo 4 processado
Arquivo 5 processado


In [115]:
nascidos_total = nascidos_vivos[["cod_municipio_6", "nascidos_vivos"]].groupby("cod_municipio_6", as_index=False).sum()
nascidos_total.rename(columns={"nascidos_vivos": "nascidos_totais"}, inplace=True)

In [116]:
nascidos_solteiras = nascidos_vivos.query("estado_civil == '1'")[["cod_municipio_6", "nascidos_vivos"]].groupby("cod_municipio_6", as_index=False).sum()
nascidos_solteiras.rename(columns={"nascidos_vivos": "nascidos_solteiras"}, inplace=True)

In [127]:
nascidos_relacao = nascidos_total.merge(nascidos_solteiras, on="cod_municipio_6", how="left")
nascidos_relacao["nascidos_solteiras"] = nascidos_relacao["nascidos_solteiras"].fillna(0)

In [130]:
nascidos_relacao["relacao_solteiras"] = nascidos_relacao["nascidos_solteiras"] / nascidos_relacao["nascidos_totais"]
nascidos_relacao.to_csv("C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Indicadores/terceiros/vunerabilidade_familiar.csv", index=False)

In [93]:
def idade_grupos(idade):
    if idade <= 10:
        return "0_10"
    elif idade <= 14:
        return "11_14"
    elif idade <= 19:
        return "15_19"
    else:
        return "20_"
    
nascidos_vivos_idade = nascidos_vivos.query("idade < 20")

nascidos_vivos_idade = nascidos_vivos_idade.copy()

nascidos_vivos_idade["grupo_idade"] = nascidos_vivos_idade["idade"].apply(lambda x: idade_grupos(x))

nascidos_vivos_idade = nascidos_vivos_idade[["cod_municipio_6", "grupo_idade", "nascidos_vivos"]]
nascidos_vivos_idade = nascidos_vivos_idade.groupby(["cod_municipio_6", "grupo_idade"], as_index=False).sum()
nascidos_vivos_idade = nascidos_vivos_idade.sort_values(by=["cod_municipio_6", "grupo_idade"])


nascidos_vivos_list = nascidos_vivos[["cod_municipio_6"]].drop_duplicates()

nascidos_vivos_list_1 = nascidos_vivos_list.copy()
nascidos_vivos_list_2 = nascidos_vivos_list.copy()
nascidos_vivos_list_3 = nascidos_vivos_list.copy()

nascidos_vivos_list_1["grupo_idade"] = "0_10"
nascidos_vivos_list_2["grupo_idade"] = "11_14"
nascidos_vivos_list_3["grupo_idade"] = "15_19"

nascidos_vivos_list_1["nascidos_vivos"] = 0
nascidos_vivos_list_2["nascidos_vivos"] = 0
nascidos_vivos_list_3["nascidos_vivos"] = 0

nascidos_vivos_idade_final = pd.concat([nascidos_vivos_idade, nascidos_vivos_list_1, nascidos_vivos_list_2, nascidos_vivos_list_3])

nascidos_vivos_idade_final.drop_duplicates(["cod_municipio_6", 'grupo_idade'])


In [94]:
nascidos_vivos_list = nascidos_vivos[["cod_municipio_6"]].drop_duplicates()

nascidos_vivos_list_1 = nascidos_vivos_list.copy()
nascidos_vivos_list_2 = nascidos_vivos_list.copy()
nascidos_vivos_list_3 = nascidos_vivos_list.copy()

nascidos_vivos_list_1["grupo_idade"] = "0_10"
nascidos_vivos_list_2["grupo_idade"] = "11_14"
nascidos_vivos_list_3["grupo_idade"] = "15_19"

nascidos_vivos_list_1["nascidos_vivos"] = 0
nascidos_vivos_list_2["nascidos_vivos"] = 0
nascidos_vivos_list_3["nascidos_vivos"] = 0

nascidos_vivos_idade_final = pd.concat([nascidos_vivos_idade, nascidos_vivos_list_1, nascidos_vivos_list_2, nascidos_vivos_list_3])

nascidos_vivos_idade_final.drop_duplicates(["cod_municipio_6", 'grupo_idade'])

In [132]:
csv = pd.read_csv("C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/nascidos_vivos_tratados.csv")
parquet = pd.read_parquet("C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/nascidos_vivos_tratados.parquet")


In [136]:
parquet.sort_values(by="cod_municipio")

,cod_municipio,contagem,ano
4438,1100015,489,2020
5330,1100023,1801,2020
2211,1100031,76,2020
5354,1100049,1946,2020
4497,1100056,512,2020
...,...,...,...
3027,5222005,153,2020
1960,5222054,59,2020
1201,5222203,24,2020
26,5222302,1,2020


In [133]:
csv

,cod_municipio_6,nascidos_vivos
0,110001,356
1,110002,1544
2,110003,65
3,110004,1360
4,110005,249
...,...,...
5565,522200,206
5566,522205,97
5567,522220,64
5568,522230,69
